In [ ]:
from enum import Enum   

class System(Enum):
    PI3 = 1
    PI4 = 2
    FDP = 3

system_in_use = System.PI3

Create verification object

In [1]:
backend_cs_dll = 'table_verification_cs/bin/Debug/'
sys.path.append(backend_cs_dll)

import clr
clr.AddReference('table_verification_cs')

from TableVerification import TableVerifier, FailingProfile

tv = TableVerifier("pi3b_asbuilt_pfc17500ab_2022-06-09", "B161087,B211100,B261087,B291060,B215008,B261008")
print(f"Read {tv.GetNumRowsInDatatable()} Rows")


Read 1728457 Rows


Data Verification

In [ ]:
import clr

clr.AddReference('System')
from System import String, Double
from System.Collections.Generic import Dictionary

import os, sys
import matplotlib.pyplot as plt

backend_cs_dll = '/nfs/home/brendan.posehn/ws/table_verification/table_verification_cs/bin/Debug/' # get some better way to do this 
sys.path.append(backend_cs_dll)
clr.AddReference('table_verification_cs')

from TableVerification import TableVerifier, FailingProfile

fp = FailingProfile()
fp.ProfileColumnName = "B161087"
fp.SearchAxis = "NevinsA"

tbl_params = Dictionary[String, Double]()
tbl_params['psieq_soak'] = 0.08
tbl_params['beta_pol1_setpoint'] = 0.1
tbl_params['psieq_dc'] = -0.08
tbl_params['NevinsC'] = 0.44
tbl_params['NevinsN'] = 99
tbl_params['CurrentRatio'] = 0.5
fp.TableParams = tbl_params

fp.OffendingValue = 1

offending_vals = [0, 1, 2] # get this from tableaxes later on

sys.path.append(os.getenv("AURORA_REPOS"))

from flagships.post_processing.GenerateColumns import get_full_file_path
from flagships.post_processing.ParseFlagshipsFile import FlagshipsParser
from flagships.post_processing.EquilibriumPlotting import PlotPsi5PctContours

i = 1
plt.figure(figsize=(12, 6))
for val in offending_vals:
    tbl_params[fp.SearchAxis] = val
    filepath = get_full_file_path("\\mnt\\lut\\Flagships\\pi3b\\asbuilt\\pfc17500ab\\2022-06-09\\equil", "", ".2022-06-09",
            soak = tbl_params['psieq_soak'], beta=tbl_params['beta_pol1_setpoint'], dc=tbl_params['psieq_dc'], NevinsC=tbl_params['NevinsC'], 
            NevinsAB=tbl_params['NevinsA'], NevinsN=tbl_params['NevinsN'], current_ratio=tbl_params['CurrentRatio'])
    filepath = filepath.replace('\\', os.path.sep)
    if(not os.path.exists(filepath)):
        filepath = filepath[:filepath.find('ab', filepath.find('equil'))+3] + filepath[filepath.find('ab', filepath.find('equil'))+4:]
    if(not os.path.exists(filepath)):
        filepath = filepath[:filepath.rfind('ab')+3] + filepath[filepath.rfind('ab')+4:]
    if(not os.path.exists(filepath)):
        print(f'File cannot be found:\n {filepath}')
        continue
    parser = FlagshipsParser(filepath, filepath)
    plt.subplot(1, len(offending_vals), i)
    i += 1
    PlotPsi5PctContours(parser)
    plt.title(fp.SearchAxis + f" = {val}")


1. Check for holes in the data

2. Check for data discontinuities

In [ ]:
import os
import sys

print(os.getenv("AURORA_REPOS"))
sys.path.append(os.getenv("AURORA_REPOS"))

from flagships.post_processing import GenerateColumns

if(system_in_use == System.PI3):
    csv_generator =  GenerateColumns.PI3CSVGenerator()
elif(system_in_use == System.PI4):
    csv_generator =  GenerateColumns.PI4CSVGenerator()
elif(system_in_use == System.FDP):
    csv_generator = GenerateColumns.FDPCase22a_CSVGenerator()
else:
    print('Ensure correct system is set')

other_cols = ['WBPol', 'q020', 'q050', 'q080', 'WBPolNoDCInFC', 'phiPlInFC']
b_col_names = []
for val in csv_generator.add_bprobe_columns([]):
    b_col_names.append('B' + val[0])

profile_columns_of_interest = other_cols + b_col_names
existent_columns_of_interest = []

all_cols = csv_generator.create_all_columns()
all_col_names = [col.Name for col in all_cols]

for col_of_interest in profile_columns_of_interest:
    if(all_col_names.count(col_of_interest) != 0):
        existent_columns_of_interest.append(col_of_interest)

print(existent_columns_of_interest)

In [ ]:
error_profiles = ""
error_profile_lines = error_profiles.splitlines()

for line in error_profile_lines:
    table_axes_values = line[line.find('(')+1:line.find(')')].split(',')
    values = line.split(',')
    column_name = values[0]
    search_axis = values[1]
